In [ ]:
import sys, os
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), '../')))
from stochss_compute.cloud.ec2 import EC2Cluster, p

In [ ]:
cluster = EC2Cluster()

In [ ]:
# cluster.create_root_key()
vpcId = cluster.create_default_vpc()
print(f'vpcId: {vpcId}')


In [ ]:
subnetId = cluster.create_default_subnet(vpcId)
print(f'subnetId: {subnetId}')


In [ ]:
subnet = cluster.resources.Subnet('subnet-004b15f5bc579c8a9')

In [ ]:
subnet.map_public_ip_on_launch

In [ ]:
sgId = cluster.create_default_security_group(vpcId)
print(f'sgId: {sgId}')


In [ ]:
head = cluster.launch_instances(subnetId=subnetId, securityGroupId=sgId)
print(f'head: {head}')


In [ ]:
head.authorize_SSH()

In [ ]:
cluster.terminate_all_instances()